## Estudo de Caso mRNA e lncRNA 

### Extração de características com k-mer 
- Indormações sobre o K-mer: http://rosalind.info/problems/kmer/
- Código Python: https://github.com/Bonidia/kmerSchemaPython

- Download das bases mRNA.fasta e lncRNA.fasta -   https://github.com/Bonidia/FeatureExtraction_BiologicalSequences/tree/master/examples


- Exemplo de uso: Extração com findKmers.py: <br>
    * (i) python3 findKmers.py -i lncRNA.fasta -o lncRNA.csv -l 1 -k 3 
    * (ii)python3 findKmers.py -i mRNA.fasta -o mRNA.csv -l 0 -k 3


#### Divisão da base em treino e teste

In [1]:
#Base lncRNA e mRNA
import os 
import pandas
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (recall_score,
                             accuracy_score,
                             precision_score,
                             f1_score)

def split(finput, test_rate):
    dataset = pandas.read_csv(finput)
    X = dataset.iloc[:, :-1]
    y = dataset.iloc[:, -1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_rate)
    train = pandas.concat([X_train, y_train], axis=1)
    test = pandas.concat([X_test, y_test], axis=1)
    
    trainData = os.path.splitext(finput)[0]+"_train"+os.path.splitext(finput)[1]
    testData = os.path.splitext(finput)[0]+"_test"+os.path.splitext(finput)[1]
    train.to_csv(trainData, index=False)
    test.to_csv(testData, index=False)    
    return                             


# Aplica a divisão treino e teste nas bases mRNA e lncRNA

split('mRNA.csv',0.3)
split('lncRNA.csv',0.3)

# carrega a base de dados treino lncRNA e mRNA
lncRNA_data = pandas.read_csv('bases/lncRNA_train.csv')
mRNA_data = pandas.read_csv('bases/mRNA_train.csv')
dadosTreino = pandas.concat([lncRNA_data,mRNA_data])

#Divide a base entre os previsores e classe
colunas = dadosTreino.columns.drop('class')

# Gera os previsores e classe (X e y)
X = dadosTreino[colunas].values
y = dadosTreino['class']

## Gera os dados de teste
# carrega a base de dados teste lncRNA e mRNA
lncRNA_test = pandas.read_csv('bases/lncRNA_test.csv')
mRNA_test = pandas.read_csv('bases/mRNA_test.csv')
dados = pandas.concat([lncRNA_test,mRNA_test])

dadosTeste = dados.columns.drop('class')
X_teste = dados[dadosTeste].values
y_teste = dados['class']

#### Atributos (k-mers) utilizados 

In [2]:
# Exibe a quantidade de atributos
print("Columns size >>> %d"%len(colunas))

# Exibe o nome dos atributos
print(dadosTreino.columns)

Columns size >>> 84
Index(['A', 'C', 'G', 'T', 'AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT',
       'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT', 'AAA', 'AAC', 'AAG',
       'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA',
       'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG',
       'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA',
       'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG',
       'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA',
       'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG',
       'TTT', 'class'],
      dtype='object')


In [3]:
print(X.shape, y.shape, X_teste.shape, y_teste.shape)

(3556, 84) (3556,) (1524, 84) (1524,)


#### Aplica o modelo de predição com Decision Tree

In [4]:
# instancia um DecisionTreeClassifier
clf = DecisionTreeClassifier()
# treina o knn
clf.fit(X, y)

y_pred = clf.predict(X_teste)
#print(y_pred)

 # gerar score baseado na acurácia
acuracidade = round(accuracy_score(y_teste,y_pred)*100,2)
print(acuracidade)

86.81


## Seleção de características com Wrapper e Recursive feature elimination
#### Detalhes: https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection

In [ ]:
print(X.shape)

In [5]:
from sklearn.feature_selection import RFE

# instancia um DecisionTreeClassifier
clf = DecisionTreeClassifier()
selector = RFE(clf, step=1)
selector = selector.fit(X, y)

#Armazena a nova dimensão do vetor de características
features = selector.fit_transform(X,y)

# instancia um DecisionTreeClassifier
y_pred = selector.predict(X_teste)

print(selector.ranking_)
print(selector.get_support(indices=True))

 # gerar score baseado na acurácia
acuracidade = round(accuracy_score(y_teste,y_pred)*100,2)
print(acuracidade)



[43  3  1 28  1 38  1  7 27 22 32  9 35 17 19 11 26 21 23  1 41  1  1  1
  1  1 12 30  1  1  5  1  1  4  1 10  1 15  1  1 14 40 36 34 25  1  1 39
  1  1  1 33 16  1 29  8 24 18  1 31  1  1  1  1  1  1 20  1  1  1  1  1
  1  1 13  2 42  6  1  1  1  1 37  1]
[ 2  4  6 19 21 22 23 24 25 28 29 31 32 34 36 38 39 45 46 48 49 50 53 58
 60 61 62 63 64 65 67 68 69 70 71 72 73 78 79 80 81 83]
86.68


In [6]:
print(features.shape)

(3556, 42)


### Obtendo o nome dos 42 atributos

In [7]:
temp = pandas.Series(selector.support_,index = colunas)
wrapperApproach = temp[temp==True].index
print(wrapperApproach)

Index(['G', 'AA', 'AG', 'TT', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'AGA', 'AGC',
       'AGT', 'ATA', 'ATG', 'CAA', 'CAG', 'CAT', 'CGC', 'CGG', 'CTA', 'CTC',
       'CTG', 'GAC', 'GCG', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTT',
       'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TGG', 'TGT', 'TTA', 'TTC',
       'TTT'],
      dtype='object')


In [8]:
# Deixa no formato list
featuresList = wrapperApproach.tolist()

In [9]:
print(featuresList)

['G', 'AA', 'AG', 'TT', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'AGA', 'AGC', 'AGT', 'ATA', 'ATG', 'CAA', 'CAG', 'CAT', 'CGC', 'CGG', 'CTA', 'CTC', 'CTG', 'GAC', 'GCG', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTT']


## Seleção de atributos abordagem filtro

In [10]:
#Feature selection with Mutual Information 
import numpy
from pandas.plotting import scatter_matrix
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif

#selector = SelectKBest(score_func=chi2, k=42)
selector = SelectKBest(score_func=mutual_info_classif, k=42)
#selector = SelectKBest(score_func=f_classif, k=42)

selector.fit(X, y)
# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
print(cols)

[ 2  3  6 12 13 14 16 19 20 21 23 29 30 31 32 33 35 37 38 39 41 44 45 46
 50 54 55 58 59 60 61 62 63 64 66 68 70 71 72 78 80 83]


### Nome dos atributos

In [11]:
temp = pandas.Series(selector.get_support(indices=False),index = colunas)
filterApproach = temp[temp==True].index
print(filterApproach)

Index(['G', 'T', 'AG', 'GA', 'GC', 'GG', 'TA', 'TT', 'AAA', 'AAC', 'AAT',
       'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATT', 'CAC', 'CAG', 'CAT', 'CCC',
       'CGA', 'CGC', 'CGG', 'CTG', 'GAG', 'GAT', 'GCG', 'GCT', 'GGA', 'GGC',
       'GGG', 'GGT', 'GTA', 'GTG', 'TAA', 'TAG', 'TAT', 'TCA', 'TGG', 'TTA',
       'TTT'],
      dtype='object')


In [12]:
# Deixa no formato list
featuresList = filterApproach.tolist()

In [13]:
print(featuresList)

['G', 'T', 'AG', 'GA', 'GC', 'GG', 'TA', 'TT', 'AAA', 'AAC', 'AAT', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATT', 'CAC', 'CAG', 'CAT', 'CCC', 'CGA', 'CGC', 'CGG', 'CTG', 'GAG', 'GAT', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTG', 'TAA', 'TAG', 'TAT', 'TCA', 'TGG', 'TTA', 'TTT']


### Avaliação entre as features (Filter x Wrapper)

In [14]:
diff = list(set(wrapperApproach) - set(filterApproach))
equal = list(set(wrapperApproach).intersection(filterApproach))
print("k-mers diferentes:")
print(diff)
print("k-mers iguais:")
print(equal)

k-mers diferentes:
['TAC', 'GAC', 'TCC', 'AA', 'ACA', 'ATG', 'TTC', 'AAG', 'ACC', 'GTC', 'CAA', 'GTT', 'TGT', 'CTC', 'AGA', 'CTA']
k-mers iguais:
['ATA', 'GCG', 'AAC', 'GGA', 'TCA', 'TAA', 'GGG', 'CGC', 'CGG', 'G', 'GGC', 'TTT', 'GTA', 'TGG', 'AAT', 'CAG', 'CAT', 'TAT', 'AG', 'AGC', 'GGT', 'CTG', 'TTA', 'TT', 'TAG', 'AGT']


# Estudo de caso com o framework MathFeature
- https://bonidia.github.io/MathFeature/

Utilização por linha de comando:
- git clone https://github.com/Bonidia/MathFeature.git MathFeature
- cd MathFeature
- pip3 install -r requirements.txt
- apt-get -y install python3-igraph

## Baixando sequências de repositórios públicos
- GENCODE: https://www.gencodegenes.org/
- PHYTOZOME: https://phytozome-next.jgi.doe.gov/
- GREENC: http://greenc.sequentiabiotech.com/wiki/Main_Page

## Etapas de pré-processamento

- Contagem da quantidade de sequências;
- Amostragem;
- Remoção de redundância;
- Eliminação de ruídos.
- Extração de características;
- Composição da base de dados